<span style="color:red">**<<<<<<< local**</span>

## Spark setup

In [59]:
import coursierapi.MavenRepository
interp.repositories.update(
  interp.repositories() ::: List(MavenRepository.of("https://oss.sonatype.org/content/repositories/snapshots"))
)

import $ivy.`org.apache.spark::spark-sql:3.1.1`
import $ivy.`org.typelevel::cats-core:2.3.0`
import $ivy.`com.lihaoyi::sourcecode:0.2.6`
import $ivy.`org.hablapps::doric:0.0.0+81-92203c5b-SNAPSHOT`

import coursierapi.MavenRepository

import $ivy.$                                  

import $ivy.$                               

import $ivy.$                              

import $ivy.$                                               

In [60]:
import org.apache.spark._
import org.apache.spark.sql._
import org.apache.spark.sql.{functions => f}
import habla.doric._
import habla.doric.{functions => doricf}

import org.apache.spark._

import org.apache.spark.sql._

import org.apache.spark.sql.{functions => f}

import habla.doric._

import habla.doric.{functions => doricf}

In [61]:
val spark = org.apache.spark.sql.SparkSession.builder().appName("test").master("local").getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

spark: SparkSession = org.apache.spark.sql.SparkSession@1f28f489

In [62]:
import spark.implicits._

import spark.implicits._

## Typed join expressions

Spark column expressions are subject to the same problems that we explored [previously]("README.ipynb): lack of static typing, unreported errors at DataFrame compile-time, unprotected implicit castings, etc. For instance, given the following DataFrames:

<span style="color:red">**=======**</span>

In [63]:
val leftdf = List((1, 1, "hi"), (2, 2, "bye"), (3, 3, "3")).toDF("id-left", "id", "value-left")
val rightdf = List((1, 1, "hi"), (2, 2, "bye"), (3, 3, "3")).toDF("id-right", "id", "value-right")

leftdf: DataFrame = [id-left: int, id: int ... 1 more field]
rightdf: DataFrame = [id-right: int, id: int ... 1 more field]

the following equi-join expressions, where we use different mechanisms to refer to the corresponding columns, compile and run without problems:

In [45]:
leftdf.join(rightdf, f.col("id-left") === f.col("id-right"))
leftdf.join(rightdf, leftdf("id") === rightdf("id"))
leftdf.alias("left").join(rightdf.alias("right"), f.col("left.id") === f.col("right.id"))

res44_0: DataFrame = [id-left: int, id: int ... 4 more fields]
res44_1: DataFrame = [id-left: int, id: int ... 4 more fields]
res44_2: DataFrame = [id-left: int, id: int ... 4 more fields]

However, if some column name or type is wrong, then errors will be shown too late, or implicit type casts will be applied. For instance, the following DataFrame compiles in Spark and runs with garbage results:

In [46]:
val dfjoin = leftdf.join(rightdf, leftdf("id") === rightdf("value-right"))

dfjoin: DataFrame = [id-left: int, id: int ... 4 more fields]

In [47]:
dfjoin.show

+-------+---+----------+--------+---+-----------+
|id-left| id|value-left|id-right| id|value-right|
+-------+---+----------+--------+---+-----------+
|      3|  3|         3|       3|  3|          3|
+-------+---+----------+--------+---+-----------+



Using doric, errors can be detected at (Scala) compile-time:

In [63]:
// Scala will prevent this from compiling successfully
def dfjoin = leftdf.join(rightdf, LeftDF.colInt("id") === RightDF.colString("value-right"), "inner")

cmd63.sc:1: type mismatch;
 found   : habla.doric.RightDoricColumn[String]
 required: habla.doric.RightDoricColumn[Int]
def dfjoin = leftdf.join(rightdf, LeftDF.colInt("id") === RightDF.colString("value-right"), "inner")
                                                                           ^Compilation Failed

: 

or at DataFrame-construction time:

In [64]:
val dfjoin = leftdf.join(rightdf, LeftDF.colInt("id-left") === RightDF.colInt("value-right"), "inner")

: 

In [65]:
val dfjoin = leftdf.join(rightdf, LeftDF.colInt("id-left1") === RightDF.colInt("value-right"), "inner")

: 

As you can see, join doric expressions also enjoy all the goodies concerning error location :D

If everything is well-typed, then the DataFrame constructed will be exactly the same than the one obtained using conventional column expressions:

In [66]:
leftdf.join(rightdf, LeftDF.colInt("id") === RightDF.colInt("id"), "inner")

res65: DataFrame = [id-left: int, id: int ... 4 more fields]

As you can see, _join_ doric column expressions refer to the left and right DataFrames using the special objects `LeftDF` and `RightDF`, respectively. We can also refer to arbitrarily complex column expressions within the context of the left and right DataFrames, enclosing the expression between parenthesis. For instance:

In [69]:
val leftCol = LeftDF.colString("id2")
val rightCol = RightDF(/* complex column expression here*/ colInt("id").cast[String])
leftdf.join(rightdf, leftCol === rightCol, "inner").show

: 

and, as this example also shows, we can also decompose doric join expressions in different functions so as to obtain more modular designs.

<span style="color:red">**>>>>>>> remote**</span>

In [ ]:
<<<<<<< local
val leftdf = List((1, 1, "hi"), (2, 2, "bye"), (3, 3, "3")).toDF("id-left", "id", "value-left")
val rightdf = List((1, 1, "hi"), (2, 2, "bye"), (3, 3, "3")).toDF("id-right", "id", "value-right")
=======
val leftdf2 = List(("1","hi"), ("2", "bye")).toDF("id", "value-left")
val rightdf2 = List((1,"hi"), (2, "bye")).toDF("id", "value-right")
>>>>>>> remote

<span style="color:red">**<<<<<<< local**</span>

the following equi-join expressions, where we use different mechanisms to refer to the corresponding columns, compile and run without problems:

In [45]:
leftdf.join(rightdf, f.col("id-left") === f.col("id-right"))
leftdf.join(rightdf, leftdf("id") === rightdf("id"))
leftdf.alias("left").join(rightdf.alias("right"), f.col("left.id") === f.col("right.id"))

res44_0: DataFrame = [id-left: int, id: int ... 4 more fields]
res44_1: DataFrame = [id-left: int, id: int ... 4 more fields]
res44_2: DataFrame = [id-left: int, id: int ... 4 more fields]

However, if some column name or type is wrong, then errors will be shown too late, or implicit type casts will be applied. For instance, the following DataFrame compiles in Spark and runs with garbage results:

In [46]:
val dfjoin = leftdf.join(rightdf, leftdf("id") === rightdf("value-right"))

dfjoin: DataFrame = [id-left: int, id: int ... 4 more fields]

In [47]:
dfjoin.show

+-------+---+----------+--------+---+-----------+
|id-left| id|value-left|id-right| id|value-right|
+-------+---+----------+--------+---+-----------+
|      3|  3|         3|       3|  3|          3|
+-------+---+----------+--------+---+-----------+



Using doric, errors can be detected at (Scala) compile-time:

In [63]:
// Scala will prevent this from compiling successfully
def dfjoin = leftdf.join(rightdf, LeftDF.colInt("id") === RightDF.colString("value-right"), "inner")

cmd63.sc:1: type mismatch;
 found   : habla.doric.RightDoricColumn[String]
 required: habla.doric.RightDoricColumn[Int]
def dfjoin = leftdf.join(rightdf, LeftDF.colInt("id") === RightDF.colString("value-right"), "inner")
                                                                           ^Compilation Failed

: 

or at DataFrame-construction time:

In [64]:
val dfjoin = leftdf.join(rightdf, LeftDF.colInt("id-left") === RightDF.colInt("value-right"), "inner")

: 

In [65]:
val dfjoin = leftdf.join(rightdf, LeftDF.colInt("id-left1") === RightDF.colInt("value-right"), "inner")

: 

As you can see, join doric expressions also enjoy all the goodies concerning error location :D

If everything is well-typed, then the DataFrame constructed will be exactly the same than the one obtained using conventional column expressions:

In [66]:
leftdf.join(rightdf, LeftDF.colInt("id") === RightDF.colInt("id"), "inner")

res65: DataFrame = [id-left: int, id: int ... 4 more fields]

As you can see, _join_ doric column expressions refer to the left and right DataFrames using the special objects `LeftDF` and `RightDF`, respectively. We can also refer to arbitrarily complex column expressions within the context of the left and right DataFrames, enclosing the expression between parenthesis. For instance:

In [69]:
val leftCol = LeftDF.colString("id2")
val rightCol = RightDF(/* complex column expression here*/ colInt("id").cast[String])
leftdf.join(rightdf, leftCol === rightCol, "inner").show

: 

and, as this example also shows, we can also decompose doric join expressions in different functions so as to obtain more modular designs.

<span style="color:red">**=======**</span>

<span style="color:red">**>>>>>>> remote**</span>